## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-05-50-28 +0000,wapo,Live updates: Conservative activist Charlie Ki...,https://www.washingtonpost.com/nation/2025/09/...
1,2025-09-11-05-50-28 +0000,wapo,"At a Capitol Hill church, mourners gather at a...",https://www.washingtonpost.com
2,2025-09-11-05-46-00 +0000,wsj,Philippines Central Bank Doesn’t Try to Move i...,https://www.wsj.com/economy/central-banking/ph...
3,2025-09-11-05-43-38 +0000,nyt,Hopes for Fast Capture of Charlie Kirk’s Shoot...,https://www.nytimes.com/2025/09/11/us/charlie-...
4,2025-09-11-05-40-57 +0000,bbc,Killing of Trump ally lays bare America's bloo...,https://www.bbc.com/news/articles/cm2z9z4m22ro...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2310/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
5,kirk,48
34,trump,47
4,charlie,46
273,new,26
66,shooting,17


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
75,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,200
19,2025-09-11-04-12-02 +0000,nypost,Kai Trump reacts to death of ‘close family fri...,https://nypost.com/2025/09/11/us-news/charlie-...,179
148,2025-09-10-20-36-13 +0000,nyt,Trump and Leaders Across Political Spectrum Re...,https://www.nytimes.com/2025/09/10/us/charlie-...,173
13,2025-09-11-04-30-20 +0000,bbc,Charlie Kirk shooting: What we know about deat...,https://www.bbc.com/news/articles/cy04p4x21e5o...,172
161,2025-09-10-20-05-36 +0000,latimes,Charlie Kirk was a force with young conservati...,https://www.latimes.com/politics/story/2025-09...,171


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
75,200,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...
233,94,2025-09-10-17-08-00 +0000,wsj,Former Vice President Kamala Harris said in a ...,https://www.wsj.com/politics/elections/harris-...
147,65,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
334,60,2025-09-10-10-00-00 +0000,nypost,Lisa Cook should be kicked out of Fed’s Septem...,https://nypost.com/2025/09/10/us-news/lisa-coo...
289,58,2025-09-10-13-57-09 +0000,wapo,Israel escalates attacks on Gaza City as Qatar...,https://www.washingtonpost.com/world/2025/09/1...
23,56,2025-09-11-03-58-18 +0000,nypost,Innocent 17-year-old girl dies weeks after she...,https://nypost.com/2025/09/10/us-news/innocent...
360,46,2025-09-10-08-17-33 +0000,nypost,South Korea sends plane to US to bring back wo...,https://nypost.com/2025/09/10/world-news/south...
65,44,2025-09-11-00-22-00 +0000,wsj,Senate Republicans voted down a measure to rel...,https://www.wsj.com/politics/policy/senate-rep...
275,44,2025-09-10-15-04-00 +0000,wsj,If the Supreme Court rules in favor of the pre...,https://www.wsj.com/economy/trade/tariff-case-...
363,41,2025-09-10-08-07-20 +0000,nyt,Before and after photos show how protests dama...,https://www.nytimes.com/live/2025/09/10/world/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
